# Read mapping and differential coverage

## Intoduction

As per the general prokaryote metagenomics pipeline materials, now that we have a final set of dereplicated genomes (full and/or partial) we can calculate differntial coverage profiles across samples by mapping sequencing reads for each sample back to the dereplicated genome set.

Mapping reads derived from DNA will give genomic coverage profiles, while mapping RNAseq reads will give gene transcriptional profiles.

There are several common options available for read mapping software (e.g. *Bowtie*, *Bowtie2*, *BBMap*), each with slightly different behaviour. Below is an example using *BBMap*.

In the example that follows, we will map the **trimmed and filtered sequencing reads** (the data set used as input into the assembly step) back to the dereplicated set of viral contigs (together with an associated prokaryote metagenome-assembled genome (MAGs) set, if available).

NOTE:

- If you also have a dereplicated set of prokaryote metagenome-assembled genomes (MAGs) from the same data set, it can be valuable to first collate the viral and prokaryote genomes together to then generate the read mapping index. This can be important especially in the case where viral genomes contain genes that are very similar or identical to those also found in prokaryote genomes (such as host-derived auxiliary metabolic genes). If prokaryote MAGs or contigs are not also included here, reads that are actually derived from the prokaryote genome abundance or transcription can be erroneously mapped to the viral genomes, falsely inflating their coverage.
  - One example from our experience: photosynthesis genes found in cyanophage can falsely have RNAseq reads mapped to them that were in fact derived from cyanobacterial photosynthetic gene transcription.
  - Also note that even by including both prokaryote and viral genomic material together in read mapping, in cases where the genes are *identical* in both, mapped reads can be randomly split between the two (depending on read mapping settings). In this case, the viral gene coverage (genomic and/or transcriptional) could once again be falsely inflated. In such cases it pays to be mindful of results and ground-truth them to other sources (for example, comparing transcription and/or genomic coverage of genes of interest with patterns across the rest of the genome in question, etc.)
- For data generated from sequecing of prokaryote isolate cultures, the read mapping process here likely gives you less ecologically relevant information compared with environmental metagenomics data, but can still give you an indication if, for example, viruses are replicating at the time of sampling (i.e. if coverage of viral genomic regions is considerably higher than the rest of the host genome). If this is of interest, run this separately for each isolate and their suite of associated virus(es) (i.e. map reads from one isolate to a file of concatenated viral sequences identified in *that* isolate (e.g. the checkv-filtered output from `virome_per_sample_derep.py`) together with the assembled isolate genome). 

***

## Data prep: collate with prokaryote data

If you have dereplicated prokaryote data (such as metagenome-assembled genomes) available from the same data set, you can concatenate these into one fasta file together with the viral contigs prior to generating the read mapping index.

For example:

In [ ]:
cd /working/dir
mkdir -p 3.read_mapping

cat 1.viral_identification/6.checkv_vOTUs/vOTUs.checkv_filtered.fna /path/to/bin_files/*.fna > 3.read_mapping/DNAviral_and_prokBIN_contigs.fna


***

## *BBMap*: build reference index



In [ ]:
#!/bin/bash
#SBATCH -A your_project_account
#SBATCH -J 5_wgs_read_mapping_index
#SBATCH --time 00:05:00
#SBATCH --mem 12GB
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 30
#SBATCH -e 5_wgs_read_mapping_index.err
#SBATCH -o 5_wgs_read_mapping_index.out

# Load dependencies
module purge
module load BBMap/38.90-gimkl-2020a

# Set up working directory
cd /working/dir/3.read_mapping

# Build index 
bbmap.sh -Xmx12g ref=DNAviral_and_prokBIN_contigs.fna


***

## WGS read mapping (genome coverage)

In the example below, we will first map the trimmed and filtered sequencing reads to the mapping index created above. The `covstats` and `statsfile` can be helpful outputs, so have been included here. But the key outputs are the `.sam` files.

Here, we will also run `pileup.sh` on the `.sam` output files to generate the `rpkm=...covstats_pileup.txt` output file. In the subsequent step, we can pass this *pileup* output file to the script `summarise_counts.py` to calculate normalised differential coverage profiles across samples. 

NOTE: 

- the read directory (`READ_DIR` variable in the script below) here points to the `1a.QC_Filtered_trimmomatic/` data. If you also included the subsequent host filtering step, this would instead point to `1b.QC_Filtered_host/` (and input file names would have to be modified accordingly)

In [ ]:
#!/bin/bash
#SBATCH -A your_project_account
#SBATCH -J 5_wgs_read_mapping
#SBATCH --time 02:30:00
#SBATCH --mem 100GB
#SBATCH --ntasks 1
#SBATCH --array=1-9
#SBATCH --cpus-per-task 30
#SBATCH -e 5_wgs_read_mapping_%a.err
#SBATCH -o 5_wgs_read_mapping_%a.out

# Load dependencies
module purge
module load BBMap/38.90-gimkl-2020a

# Set up working directories
cd /working/dir/3.read_mapping
mkdir -p WGS

READ_DIR=/path/to/wgs/1a.QC_Filtered_trimmomatic

# Run read mapping
srun bbmap.sh \
t=30 -Xmx130g ambiguous=best minid=0.95 \
in1=${READ_DIR}/S${SLURM_ARRAY_TASK_ID}_R1.fastq \
in2=${READ_DIR}/S${SLURM_ARRAY_TASK_ID}_R2.fastq \
covstats=WGS/S${SLURM_ARRAY_TASK_ID}.covstats.txt \
statsfile=WGS/S${SLURM_ARRAY_TASK_ID}.statsfile.txt \
out=WGS/S${SLURM_ARRAY_TASK_ID}.sam

# Pileup
pileup.sh \
in=WGS/S${SLURM_ARRAY_TASK_ID}.sam \
rpkm=WGS/S${SLURM_ARRAY_TASK_ID}.covstats_pileup.txt


## Calculate (normalised) differential coverage across samples

We can now compile a counts table of read counts per contig per sample using the script `summarise_counts.py`. 

This script returns sets of columns of raw and normalised read counts per sample for each contig included in the read mapping index. Normalisation can be calculated based on *total* sequencing read depth per sample or by *mapped* read counts, and outputs include TPM, RPKM, and (optionally) TMM. It is up to the user to determine the most appropriate normalisation method for your data set. 

This script (and the required associated R script `summarise_counts.R`) is available in the `../scripts/` directory.

NOTE:

- There can be many options available for this step in the process. `summarise_counts.py` was generated in our group to handle this in an automated manner, but feel free to experiment with other options
- **TMM normalisation**: We can also optionally provide a **sample mapping file** (see below) of relevant sample groupings if we wish to include TMM normalisation (sample groups information is required for calucating TMM via the R package *edgeR*)
- **Genome level count summaries**: We can also optionally provide a **genome2contig lookup table** (see below) matching contig IDs to each genome to generate 'genome'-level coverage summeries (especially relevant for, e.g., prokaryote metagenome-assembled genomes which contain more than one contig per genome). 


#### Create Sample mapping file (if also calculating TMM)

Providing an optional sample mapping enables also calculating TMM normalisation (via *edgeR*). The required format of the mapping file is two columns: 

1. `sampleID`: Filename substrings (one per file) that identify the sample (e.g. S1.covstats_pileup.txt substring could be `S1`). (NOTE: if a wildcard is included in the python call, this must not include any of the substring after the wildcard)
1. `group`: Group or category that each sample belongs to
1. (OPTIONAL) `lib.size`: Optional third column of read counts per sample. This is required for normalising by total library size when inputting featurecounts data (e.g. for read mapping coverage for individual genes). When using *pileup* output (e.g. for wgs (genome) coverage), the script automatically retrieves this information from the `...covstats_pileup.txt` files.

For example: 

| sampleID | group |
| :--: | :--: |
| S1 | Water_Fresh |
| S2 | Water_Fresh |
| S3 | Water_Brackish |
| S4 | Water_Brackish |
| S5 | Water_Brackish |
| S6 | Water_Brackish |
| S7 | Water_Brackish |
| S8 | Water_Marine |
| S9 | Water_Marine |

A simple bash script example has been provided below to automate this process. (In this example, total library size is also extracted from the `...covstats_pileup.txt` files to include in the mapping file, but this is not essential). But note that this can sometimes be easier to simply manually create the mapping file (e.g. via *nano* or some equivalent).

In [ ]:
cd /working/dir

# Generate sample mapping file
echo -e "sampleID\tgroup\tlib.size" > 3.read_mapping/WGS/wgs_sample_mapping_file.txt
for filename in $(ls 3.read_mapping/WGS/*covstats_pileup.txt); do
    entry=$(echo ${filename} | sed -e 's/\(S[0-9]*\)\..*/\1/g' -e 's/.*\/\(.*\)/\1/g')
    if [[ ${entry} == *"S1"* ]] || [[ ${entry} == *"S2"* ]]; then
        group='Water_Fresh'
    elif [[ ${entry} == *"S3"* ]] || [[ ${entry} == *"S4"* ]] || [[ ${entry} == *"S5"* ]] || [[ ${entry} == *"S6"* ]] || [[ ${entry} == *"S7"* ]]; then
        group='Water_Brackish'
    elif [[ ${entry} == *"S8"* ]] || [[ ${entry} == *"S9"* ]]; then
        group='Water_Marine'
    fi
    libsize=$(grep '#Reads' ${filename} | sed 's/#Reads\s//')
    echo -e "${entry}\t${group}\t${libsize}" >> 3.read_mapping/WGS/wgs_sample_mapping_file.txt
done


#### Create genome2contig_lookupTable.tsv

A lookup table matching contig IDs to separate genomes can be useful, particularly in the case of binned sets of contigs (e.g. metagenome-assembled genomes). Including this table in the subsequent `summarise_counts.py` step also generates 'genome' level summaries of read coverage (rather than just contig-level coverage stats).

NOTE: This may ultimately be put into a script for ease of use. But for now we can use the python code below to automatically generate a single lookup table for both the viral contig data set and binned prokaryote metagenome-assembled genomes (if available).

In [ ]:
cd /working/dir

# Load python
module purge
module load Python/3.8.2-gimkl-2020a
python3
### Import required libraries
import pandas as pd
import numpy as np
from Bio.SeqIO.FastaIO import SimpleFastaParser
import re
import os

# Compile bin2contig table
with open('3.read_mapping/genome2contig_lookupTable.tsv', 'w') as write_file:
    # header
    write_file.write('binID\tcontigID\n')
    # viral contigs
    with open('1.viral_identification/6.checkv_vOTUs/vOTUs.checkv_filtered.fna', 'r') as read_fasta:
        for name, seq in SimpleFastaParser(read_fasta):
            write_file.write(name + '\t' + name + '\n')
    # Prok bins
    binfiles_directory = os.fsencode('/path/to/bin_files')
    for file in os.listdir(binfiles_directory):
        filename = os.fsdecode(file)
        binID = os.path.splitext(filename)[0]
        with open('/path/to/bin_files/' + filename, 'r') as read_fasta:
            for name, seq in SimpleFastaParser(read_fasta):
                write_file.write(binID + '\t' + name + '\n')

quit()


#### Run summarise_counts.py

NOTE:

- `--sample_mapping_file` and `--genome_mapping_file` are  optional. If including the former, TMM normalisation will also be calculated. If including the latter, a genome-level summary file will also be generated (read counts per all contigs in each *genome* rather than per contig). The genome summary file includes summed raw counts, as well normalisation based on library size (total or mapped, depending on the setting) and genome size (length of all included contigs).
- `--genome_libSize_norm` determines whether to normalise to the *minimum* read depth of any of the samples, or to the *mean* read depth.
- `--lib_norm` sets whether to normalise based on total read depth per sample or mapped reads depth.
- `--count_threshold` sets a filter that zeros out raw read counts for any contig in a given sample that fall below this threshold (to help filter out spurious read mapping). You can experiment with this to determine what seems appropriate for your data.
- `--read_counts` outputs a summary table of total and mapped read counts per sample for future reference, and `--output` is the final differential coverage count table (including multiple normalisation metrics)
  - n.b. if a genome_mapping_file is included, the genome level summary takes this file name and appends 'genomeSummary'

In [ ]:
cd /working/dir

# Load python and R
module purge
module load Python/3.8.2-gimkl-2020a
module load R-bundle-Bioconductor/3.15-gimkl-2022a-R-4.2.1

# Run summarise_counts.py 
# Note: the quotes in the --input line are required to enable the wildcard (*) to be interpreted correctly
/path/to/scripts/summarise_counts.py \
    --input '3.read_mapping/WGS/*.covstats_pileup.txt' \
    --format pileup \
    --sample_mapping_file 3.read_mapping/WGS/wgs_sample_mapping_file.txt \
    --genome_mapping_file 3.read_mapping/genome2contig_lookupTable.tsv \
    --genome_libSize_norm min \
    --lib_norm total \
    --count_threshold 10 \
    --read_counts 3.read_mapping/WGS/wgs_summary_read_counts.tsv \
    --output 3.read_mapping/WGS/wgs_summary_count_table.tsv


***

## WTS read mapping (gene transcription)

If you also have RNAseq data available, we can follow a similar process as above to generate differential coverage of transcribed genes across samples.

Here, we can first map trimmed and filtered RNAseq reads to the same read mapping index generated above. A gene coordinates file (start and end coordinates and strandedness of predicted genes within each contig; GTF or SAF format) needs to then be generated to pass to a program such as *featureCounts* to generate a gene counts output for each sample. 

The *featureCounts* output can be passed into `summarise_counts.py` to generate a count table of normalised differential transcription coverage per gene per sample. As above, providing a genome mapping file will also output summed counts of gene transcription per genome, and providing a mapping file will also calculate TMM normalisation **as well as *edgeR* differentially expressed genes (DEG) analysis across sample groups** (Note, however, that the latter may not be entirely appropriate for metagenome RNAseq data sets (as was designed for single genome transcription (DEG) analyses), and so should be viewed with caution. 

#### WTS read mapping

Map trimmed and filtered RNAseq reads to the same read mapping reference generated above.

In [ ]:
#!/bin/bash
#SBATCH -A your_project_account
#SBATCH -J 5_wts_read_mapping
#SBATCH --time 08:00:00
#SBATCH --mem 94GB
#SBATCH --ntasks 1
#SBATCH --array=1-9
#SBATCH --cpus-per-task 30
#SBATCH -e 5_wts_read_mapping_%a.err
#SBATCH -o 5_wts_read_mapping_%a.out

# Load dependencies
module purge
module load BBMap/38.90-gimkl-2020a
module load SAMtools/1.15.1-GCC-11.3.0

# Set up working directories
cd /working/dir/3.read_mapping
mkdir -p WTS/

READ_DIR=/path/to/wts/1.trimmed_filtered

# Run read mapping
srun bbmap.sh \
t=30 -Xmx94g ambiguous=best minid=0.95 \
in1=${READ_DIR}/S${SLURM_ARRAY_TASK_ID}_R1.fastq \
in2=${READ_DIR}/S${SLURM_ARRAY_TASK_ID}_R2.fastq \
covstats=WTS/S${SLURM_ARRAY_TASK_ID}.covstats.txt \
statsfile=WTS/S${SLURM_ARRAY_TASK_ID}.statsfile.txt \
out=WTS/S${SLURM_ARRAY_TASK_ID}.sam
# convert to bam
samtools sort -@ 10 -o WTS/S${SLURM_ARRAY_TASK_ID}.bam WTS/S${SLURM_ARRAY_TASK_ID}.sam


#### In development...

A script will be developed to generate the appropriate gene coordinates file required by *featureCounts*, based on the gene calling outputs from *DRAM-v*. This is currently a work in progress. In the meantime, you will need to generate one yourself (GTF or SAF format).


#### Run *featureCounts* 

NOTE: *featureCounts* is not currently available in NeSI. You will need to install this separately.

In [ ]:
# Set up directories
cd /working/dir
mkdir -p 3.read_mapping/featureCounts

# Run featureCounts
/path/to/Software/featureCounts \
-p -T 8 -t exon -F SAF \
-a /path/to/gene_coords.SAF \
-o 3.read_mapping/featureCounts/gene_counts.txt \
3.read_mapping/WTS/*.bam


#### Generate sample mapping file

As above, this can optionally be included in `summarise_counts.py` to also generate TMM normalised coverages and also run *edgeR* differential gene expression analysis. 

The required format is as above (see WGS read mapping section). 

The bash script below is one way of generating this in a semi-automated manner, this time extracting the information (including library size) from the *BBMap* `...statsfile.txt` output file. 

Note: 

- In the case of using *featureCounts* outputs with `summarise_counts.py`, it is necessary to include the library size in the mapping file here if `--lib_norm` is set to `total` when running `summarise_counts.py`


In [ ]:
cd /working/dir

# Generate mapping file
echo -e "sampleID\tgroup\tlib.size" > 3.read_mapping/WTS/wts_sample_mapping_file.txt
for filename in $(ls 3.read_mapping/WTS/*.statsfile.txt); do
    entry=$(echo ${filename} | sed -e 's/\(S[0-9].*\)\.statsfile.txt/\1/g' -e 's/.*\/\(.*\)/\1/g')
    if [[ ${entry} == "S1"* ]] || [[ ${entry} == "S2"* ]]; then
        group='water_column_fresh'
    elif [[ ${entry} == "S3"* ]] || [[ ${entry} == "S4"* ]] || [[ ${entry} == "S5"* ]] || [[ ${entry} == "S6"* ]] || [[ ${entry} == "S7"* ]]; then
        group='water_column_brackish'
    elif [[ ${entry} == "S8"* ]] || [[ ${entry} == "S9"* ]]; then
        group='water_column_marine'
    fi
    libsize=$(grep 'Reads Used:' ${filename} | sed 's/Reads Used:\s*\([0-9]*\)\s*.*/\1/')
    echo -e "${entry}\t${group}\t${libsize}" >> 3.read_mapping/WTS/wts_sample_mapping_file.txt
done


#### Run `summarise_counts.py`

Note: we can include the sample `genome2contig_lookupTable.tsv` file generated above to also output sums of transcriptional coverage over whole genomes (in contrast to per gene).

In [ ]:
# Load python and R
module purge
module load Python/3.8.2-gimkl-2020a
module load R-bundle-Bioconductor/3.15-gimkl-2022a-R-4.2.1

# Working directory
cd /working/dir/3.read_mapping/WTS

# Run summarise_counts
summarise_counts.py \
    --input '../featureCounts/gene_counts.txt' \
    --sample_mapping_file wts_sample_mapping_file.txt \
    --genome_mapping_file ../genome2contig_lookupTable.tsv \
    --genome_libSize_norm min \
    --format featurecounts \
    --lib_norm total \
    --count_threshold 5 \
    --read_counts wts_summary_read_counts.tsv \
    --edger_out wts_summary_edgeR_glmQLFTest.tsv \
    --output wts_summary_count_table.tsv


***